In [1]:
%load_ext blackcellmagic

In [2]:
import pandas as pd
import vaex

In [3]:
#pipenv uninstall pytables

In [4]:
df = vaex.open("million_9_song(1).hdf5")

In [5]:
si = df.song_identifier.evaluate()

In [6]:
annoy_index = df.index.evaluate()

In [7]:
song_info = list(zip(si,annoy_index))

In [8]:
song_info[0]

('dangerkids-hostage', 0)

## Push to Elastic Search

In [9]:
from elasticsearch import Elasticsearch 

In [10]:
es=Elasticsearch(['https://search-music-recommender-wwryojqvlkrgqk3a5xiqtubwzy.us-east-2.es.amazonaws.com'])

In [11]:
es

<Elasticsearch([{'host': 'search-music-recommender-wwryojqvlkrgqk3a5xiqtubwzy.us-east-2.es.amazonaws.com', 'port': 443, 'use_ssl': True}])>

In [39]:
es.indices.delete(index='song-index', ignore=[400,404])

{'acknowledged': True}

In [40]:
es.indices.create(index='song-index', ignore=[400,404])

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'song-index'}

In [41]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [42]:
s1={"song-identifier":"dangerkids-hostage","annoy-id":1}
res = es.index(index='song-index', doc_type='song-doc',id=1,body=s1)

In [43]:
res

{'_index': 'song-index',
 '_type': 'song-doc',
 '_id': '1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [44]:
def song_body(body_index_tuple):
    body = {
        "_index": "song-index",
        "_type": "song-doc",
        "_id": int(body_index_tuple[1]), 
        "song-identifier": body_index_tuple[0],
        "annoy-id": int(body_index_tuple[1]),
    }
    return body

In [45]:
import warnings
warnings.filterwarnings('ignore')

## Bulk upload
So far unsuccessful

In [46]:
from elasticsearch.helpers import bulk

In [47]:
def gendata():
    mywords = ['foo', 'bar', 'baz']
    for song in song_info:
        yield song_body(song)

bulk(es, gendata())

(1844757, [])

# Testing ES

In [29]:
from elasticsearch import Elasticsearch
es=Elasticsearch(['https://search-music-recommender-wwryojqvlkrgqk3a5xiqtubwzy.us-east-2.es.amazonaws.com'])

In [30]:
 es.search(
        index="song-index",
        body={"query": {"match": {"song-identifier": "wu tang clang triumph"}}},
    )["hits"]["hits"]

[{'_index': 'song-index',
  '_type': 'song-doc',
  '_id': 'pPi-8WsBykaCkwsh2EEH',
  '_score': 28.552212,
  '_source': {'song-identifier': 'wu-tang-clan-triumph', 'annoy-id': 1655345}},
 {'_index': 'song-index',
  '_type': 'song-doc',
  '_id': 'IPW-8WsBykaCkwshbc0_',
  '_score': 22.693602,
  '_source': {'song-identifier': 'wu-tang-clan-wu-tang-master',
   'annoy-id': 1494445}},
 {'_index': 'song-index',
  '_type': 'song-doc',
  '_id': 'ru688WsBykaCkwsh5BQB',
  '_score': 22.514074,
  '_source': {'song-identifier': 'wu-tang-clan-wu-tang-reunion',
   'annoy-id': 988475}},
 {'_index': 'song-index',
  '_type': 'song-doc',
  '_id': 'L_i-8WsBykaCkwsh2Uku',
  '_score': 21.56496,
  '_source': {'song-identifier': 'wu-tang-clan-wu-tang-7th-chamber',
   'annoy-id': 1657276}},
 {'_index': 'song-index',
  '_type': 'song-doc',
  '_id': 'QvW-8WsBykaCkwshbc0_',
  '_score': 21.534573,
  '_source': {'song-identifier': 'wu-tang-clan-wu-tang-forever-intro',
   'annoy-id': 1494479}},
 {'_index': 'song-index'

In [31]:
q = [
    {
        "song-identifier": i["_source"]["song-identifier"],
        "annoy-id": i["_source"]["annoy-id"],
    }
    for i in es.search(
        index="song-index",
        body={"query": {"match": {"song-identifier": "wu tang clang triumph"}}},
    )["hits"]["hits"]
]

In [32]:
q[0]['annoy-id']

1655345

In [51]:
from annoy import AnnoyIndex

In [52]:
b = AnnoyIndex(200)
b.load('../test.ann')

True

In [53]:
z = b.get_nns_by_item(q[0]['annoy-id'], 10)

In [54]:
z

[1655345,
 1473340,
 1147912,
 1657793,
 1489749,
 1657252,
 1659510,
 1656813,
 1453730,
 192700]

In [63]:
es.get(index='song-index',doc_type='song-doc',id=192700)

{'_index': 'song-index',
 '_type': 'song-doc',
 '_id': '192700',
 '_version': 1,
 '_seq_no': 38563,
 '_primary_term': 1,
 'found': True,
 '_source': {'song-identifier': 'melvin-bliss-synthetic-substitution',
  'annoy-id': 192700}}

In [50]:
es.get(index='song-index',doc_type='song-doc',id=q[5]['annoy-id'])

{'_index': 'song-index',
 '_type': 'song-doc',
 '_id': '1494475',
 '_version': 1,
 '_seq_no': 299121,
 '_primary_term': 1,
 'found': True,
 '_source': {'song-identifier': 'wu-tang-clan-enter-the-wu-tang',
  'annoy-id': 1494475}}

In [88]:
def make_playlist(search):
    query = es.search(
        index="song-index",
        body={"query": {"match": {"song-identifier": search}}},
    )["hits"]["hits"][0]["_source"]["annoy-id"]
    query = b.get_nns_by_item(q[0]['annoy-id'], 10)
    return  list(map(lambda x: es.get(index='song-index',doc_type='song-doc',id=x)["_source"]["song-identifier"], query))

In [89]:
make_playlist("wu tang clan triumph")

['wu-tang-clan-triumph',
 'gang-of-four-hed-send-in-the-army',
 'bing-crosby-deep-in-the-heart-of-texas',
 'wu-tang-clan-wu-tang-clan-aint-nuthing-ta-fuck-wit',
 'snoop-dogg-g-funk-intro',
 'public-enemy-black-steel-in-the-hour-of-chaos',
 'mobb-deep-animal-instinct',
 'gza-4th-chamber',
 'original-broadway-cast-of-into-the-woods-second-midnight',
 'melvin-bliss-synthetic-substitution']